我们已经学习了如何创建新的类型，下一步是接收用户定义对象作为参数或者将其当作结果返回的函数。

## 时间

给出一个用户定义类型的例子。定义一个叫作 `Time` 的类，用于记录一天里的时间，并赋予小时数,分钟数和秒数的属性:

In [1]:
class Time:
    '''
    表示时间
    ----------
    属性:hour,minute,second
    '''
    def __init__(self,hour=12.0,minute=0.0,second=0.0):
        self.hour = hour
        self.minute = minute
        self.second = second

我们可以创建一个 `Time` 对象并给其属性重新赋值:

In [2]:
time = Time()
time.hour = 11
time.minute = 59
time.second = 30

编写一个函数 `print_time`，接受一个 `Time` 对象作为形参并以 `hour:minute:second` 的格式打印它:

In [3]:
def print_time(t):
    hour = t.hour
    minute = t.minute
    second = t.second
    print('%.2d:%.2d:%.2d'%(hour,minute,second))

## 纯函数

在下面几节中，我们会编写两个用来增加时间值的函数，它们展示了两种不同类型的函数:**纯函数和修改器**。它们也展示了称为 **原型和补丁(prototype and patch)** 的开发计划。这是一种应对复杂问题的方法，从一个简单的原型开始，并逐渐解决更多的复杂情况。

下面是 `add_time` 的一个简单原型:

In [16]:
def add_time(t1,t2):
    s = Time()
    s.hour = t1.hour + t2.hour
    s.minute = t1.minute + t2.minute
    s.second = t1.second + t2.second
    return s

这个函数创建一个新的 `Time` 对象，初始化它的属性，并返回这个新对象的一个引用。这被称为是一个**纯函数**，因为它除了返回一个值外，并不修改实参传入的任何对象，也没有任何如显示值或者获得用户输入之类的副作用。总的来说，**纯函数即不修改任何形参对象的函数**。**大部分纯函数都有返回值。**

为了测试该函数，创建两个 `Time` 对象:`start`,存放一个电影的开始时间;`duration` 存放电影的播放时间，这里记为 1 小时 35分钟。

`add_time` 计算处电影何时结束。

In [17]:
start = Time()
start.hour = 9
start.minute = 45
start.second = 0

In [18]:
duration = Time()
duration.hour = 1
duration.minute = 35
duration.second = 0

In [19]:
done = add_time(start,duration)
print_time(done)

10:80:00


这个结果并不是我们想要的。问题在于这个函数并没有处理好秒数或者分钟数超过 60 的情况。此时，我们需要将多余的秒数"进位"到分钟数，将多余的分钟数"进位"到小时数。给出一个改进的版本:

In [21]:
def add_time(t1,t2):
    s = Time()
    s.hour = t1.hour + t2.hour
    s.minute = t1.minute + t2.minute
    s.second = t1.second + t2.second
    
    if s.second >=60:
        s.second -= 60
        s.minute += 60
    
    if s.minute >=60:
        s.minute -= 60
        s.hour += 1
    
    return s

In [22]:
start = Time()
start.hour = 9
start.minute = 45
start.second = 0

duration = Time()
duration.hour = 1
duration.minute = 35
duration.second = 0

done = add_time(start,duration)
print_time(done)

11:20:00


虽然这个函数是正确的，但它已经开始变大了。我们会在后面看到一个更短的版本。

## 修改器

有时用函数修改传入的参数对象是很有用的。在这种情况下，修改对调用者是可见的，这样工作的函数称为**修改器(modifier)**。

函数 `increment` 给一个 `Time` 对象增加指定的秒数，可以自然地写为一个修改器。下面是初始版本:

In [23]:
def increment(time,seconds):
    time.second += seconds
    if time.second >=60:
        time.second -= 60
        time.minute += 1
    
    if time.minute >= 60:
        time.minute -= 60
        time.hour += 1

第一行进行基础操作，后面的代码处理前面的特殊情况。

但这个函数并不正确，例如，如果 `seconds` 比 60 大很多，那么只进位一次显然是不够的。当然可以使用 `while` 语句来替代 `if` 语句，但这样并不高效。因此，我们还需要对函数进行修改:

In [24]:
def increment(time,seconds):
    time.second += seconds
    if time.second >=60:
        m = time.second//60
        time.second = time.second%60
        time.minute += m
    
    if time.minute >= 60:
        h = time.minute//60
        time.minute = time.minute%60
        time.hour += h

任何可以使用修改器做到的功能都可以使用纯函数实现。事实上，有的编程语言只允许使用纯函数。有证据表明使用纯函数的程序比使用修改器的程序开发更快，错误更少。但有时使用修改器还是很方便的，并且函数式程序的运行效率不那么高。

## 原型和计划

刚才展示的开发计划称为 "原型和补丁": **对每个函数，编写一个可以进行基本计算的原型，再测试它，从中发现错误并打补丁。**

这种方法在对问题的理解并不深入时尤其有效。但增量地修正可能会导致代码过度复杂(因为它们需要处理很多特殊情况)，并且也不够可靠(因为很难知道你是否找到了所有错误)。

另一种方法是 **有规划开发(designed development)**。对问题有更高阶的理解能够让编程简单得多。在上面的问题中，如果更深入地理解，可以发现 `Time` 对象实际上是六十进制数里的三位数，`second` 属性是"个位数"，`minute` 属性是"60位数"，而 `hour` 属性是 "360位数"。

在编写 `add_time` 和 `increment` 时，我们实际上是在六十进制上进行加减，因此才需要从一位进位到另一位。

这个观察让我们可以考虑整个问题的另一种解决方法————我们可以将 `Time` 对象转换为整数，并利用计算机知道如何做整数运算的事实。

下面是一个将 `Time` 对象转换为整数的函数:

In [25]:
def time_to_int(time):
    minutes = time.hour*60 + time.minute
    seconds = minutes*60 + time.second
    return seconds

而下面是一个将整数转换回 `Time` 对象的函数(记着 `divmod` 函数将第一个参数除以第二个参数，并以元组的形式返回商和余数):

In [26]:
def int_to_time(seconds):
    time = Time()
    minutes, time.second = divmod(seconds,60)
    time.hour,time.minute = divmod(minutes,60)
    return time

我们还需要对函数进行测试。一种测试它们的方法是对很多 `x` 值检查 `time_to_int(int_to_time(x))==x`。这是一致性检验的一个例子。

一旦确认它们是正确的，就可以使用它们重写 `add_time`:

In [27]:
def add_time(t1,t2):
    seconds = time_to_int(t1) + time_to_int(t2)
    return int_to_time(seconds)

这个版本比最初版本短得多，并且也很容易检验。

从某个角度看，在六十进制和十进制之间来回转换比只处理时间更难。进制转换更加抽象; 我们对时间值的直觉更好。

但如果我们将时间看作六十进制数，并做好了编写转换函数的先期投入，就能得到一个更短，更可读，也更可靠的函数。

它也让我们今后更容易添加功能。例如，假设将两个 `Time` 对象相减来获得它们之间的时间间隔。简单的做法是使用借位实现减法。而使用转换函数则更简单，且更容易正确。

**有时候把一个问题弄得更难(更通用)反而会让它更简单(因为会有更少的特殊情况以及更少的出错机会)。**

## 调试

一个 `Time` 对象当 `minute` 和 `second` 的值在 0 到 60 之间(包含 0 但不包含 60)以及 `hour` 是正值时，是合法的。`hour` 和 `minute` 应当是整数值，但我们也许需要允许 `second` 拥有小数值。

这些需求称为**不变式**，因为它们应当总是为真。换句话说，如果它们不为真，则一定有什么地方出错了。

编写diamante来检查不变式可以帮你探测错误并找寻它们的根源。例如，你可以写一个像 `valid_time` 这样的函数，接收 `Time` 对象，并在它违反了一个不变式时，返回 `False`:

In [28]:
def valid_time(time):
    if time.hour <0 or time.minute < 0 or time.second < 0:
        return False
    if time.minute >=60 or time.second >= 60:
        return False
    return True

接着在每个函数的开头，可以检查参数，确保它们是有效的:

In [ ]:
def add_time(t1,t2):
    if not valid_time(t1) or not valid_time(t2):
        raise ValueError('invalid Time object in add_time')
    seconds = time_to_int(t1) + time_to_int(t2)
    return int_to_time(seconds)

In [29]:
def add_time(t1,t2):
    assert valid_time(t1) or not valid_time(t2):
    seconds = time_to_int(t1) + time_to_int(t2)
    return int_to_time(seconds)

`assert` 语句很有用，因为它们区分了处理普通条件的代码和检查错误的代码。